In [1]:
## import 套件
import torch
import pandas as pd
import numpy as np
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW ##新ㄉ 好像比較好
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import BertModel
from transformers import BertConfig

from matplotlib import pyplot as plt

In [2]:
## load data & path
columns_name = ['type','title','text']
dftrain = pd.read_csv('./data_after_sep/train.tsv',sep = '\t',names = columns_name)
dftest = pd.read_csv('./data_after_sep/test.tsv',sep = '\t',names = columns_name)
dfdev = pd.read_csv('./data_after_sep/dev.tsv',sep = '\t',names = columns_name)

model_path = './bert_pretrain_news/'
tokenizer = BertTokenizer.from_pretrained(model_path)

In [4]:
test = dftrain['text'].tolist()
print(test[123:133])

['謝龍介出示各種治「燒聲」及潤喉的秘方，強調雖然講到沙啞，但也只有拚了。（記者蔡文居攝）〔記者蔡文居／台南報導〕國民黨台南市黨部主委謝龍介全國走透透為藍營候選人輔選，深受韓粉的支持及喜愛，由於他演講十分賣力，邀約不斷，但這一、二天喉嚨已嚴重「燒聲」，讓不少韓粉和支持者心急又心疼，擔心接下來1個多月的選戰怎麼辦？對此，謝龍介今天表示，「沒問題，我是海軍陸戰隊的，不怕苦不怕難。」謝龍介上週三（27日）出現在台南市議會受訪時，還能侃侃而談，前天他在台中輔選時，已公開表明喉嚨已經拚到沙啞，還說在台中又吃了橘子後更慘，自嘲「這下子妥當了（意指不行了）」。昨天，謝龍介沙啞情況比前一天還要嚴重。不少韓粉和支持者得悉後，心急又心疼。紛紛在他的臉書粉絲專頁留言，請他「保重身體，加油！」「請保重身體，聽到助講到都燒聲了，真心令人感動又心疼！」還有不少粉絲也提供各種治喉嚨沙啞的偏方，希望他保重。對此，謝龍介今天在台南受訪時，出示各種治「燒聲」及潤喉的秘方，包括前台中市長胡志強給他的秘方，還有支持者給的客家菜脯秘方，另外還有止咳、清喉嚨、喉糖、感冒藥等中西藥及秘方，共6種之多。他說，因助講聲嘶力喝，加上有一點點感冒，聲音沙啞有比較嚴重。但距明年大選，「時間太短，百姓還生活在水深火熱之中，他們苦，我們不能停，必須馬不停蹄，日夜奔跑。」他說，昨天他連講了7場，今天下午還要到高雄，明天則到新北和台北市，雖然喉嚨已經講到沙啞，但也只有拚了。', '華為集團創辦人任正非接受美國《CNN》專訪，他表示孟晚舟就向小螞蟻一樣，夾在美中之間成為貿易戰的籌碼。（資料照，美聯社）〔財經頻道／綜合報導〕距離加拿大政府應美國的引渡要求，逮捕在溫哥華轉機的中國華為財務長孟晚舟正式屆滿1年，華為集團創辦人任正非1日接受美國《CNN》專訪，他表示孟晚舟就像小螞蟻一樣，夾在美中之間成為貿易戰的籌碼。任正非接受《CNN》專訪，表達對女兒1年來承受的苦難感到驕傲，他表示孟晚舟正在遭受很大的痛苦，但也將因此變得更強大；任正非形容，自己的女兒孟晚舟就像是隻小螞蟻，被困在2個強權的對抗之中。任正非指出，孟晚舟目前都在作畫和學習，孟的母親和丈夫也會定期飛往加拿大與她作伴；他也表示，這些磨難讓她和女兒更加親近，他們聊天的頻率比以前更多，有時也會傳給她在網路看到的有趣故事。任正非表示，在以前可能1整年女兒都不會打電話給他，她也不會問她過

In [7]:
dftrain = dftrain[(dftrain['type']== 0)|(dftrain['type']== 1)|(dftrain['type']== 5)]
dftest = dftest[(dftest['type']== 0)|(dftest['type']== 1)|(dftest['type']== 5)]
dftrain['type'] = dftrain['type'].replace(5,2)
dftest['type'] = dftest['type'].replace(5,2)
dftrain

,type,title,text
0,0,韓國瑜：我是高雄市長、總統候選人怎會是「草包」？,國民黨總統候選人韓國瑜今日上廣播節目接受專訪，期間針對外界給予「草包」標籤一事回應。（記者張...
4,0,韓民調跌破2成還不是谷底？林濁水疑惑草包竟撐這麼久,韓支持度掉進20%之下，但林濁水強調「肯定還不是谷底」，反而還困惑超級草包竟撐這麼久。（資料...
6,2,全瞎住持掰「灌氣」女信徒趁機按壓「那裡」1分鐘關1年2月,最高法院駁回黃姓住持上訴。（資料照）〔記者張文川／台北報導〕台中市婦人小雪（化名），3年前為...
9,1,天冷小心上腹痛醫生：恐是心肌梗塞,不少民眾低溫時會吃火鍋取暖。（記者林惠琴攝）〔記者林惠琴／台北報導〕入冬首波大陸冷氣團襲台，...
11,1,奇！台南黑面琵鷺上樹了,台南這棵木麻黃樹有10多隻黑面琵鷺，喜歡在樹上休息。（鳥友黃三明提供）〔記者楊金城／台南報導...
...,...,...,...
25531,2,新北消防世代交替六年級生接任火調科長等5重任,新北市消防局發布新接任火災調查科長莊渝淵等5名大隊長、一級主管調動。新北市消防局今天發布第四...
25534,1,武漢肺炎》「抗疫國家隊」成軍中研院：3~6月研發快篩,中研院攜手學研界籌組開放科學平台，已有15個國內單位投入，共同投入快速篩檢工具、藥物與疫苗研...
25536,1,武漢肺炎》鎮煞防疫271年歷史東林福德廟擴大發平安圓,林園區東林福德廟今年擴大平安圓發放，為地方及民眾祈福。高雄市林園區東林福德廟每逢農曆2月2日...
25541,1,抗議捷運彎進土城金城公園民眾在說明會高舉布條,民眾高舉布條抗議軌道彎進土城金城公園。捷運萬大樹林線二期工程LG11站今晚在新北市土城區公所...


In [8]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return input_id, tokentype, attentionmask, y
    def __len__(self):
        return len(self.input_ids)

In [9]:
train_x = dftrain['text'].tolist()
train_input_dict = tokenizer.batch_encode_plus(train_x,
                                              add_special_tokens = True,
                                              max_length = 512,
                                              truncation = True,                ##是否截斷
                                              return_special_tokens_mask = True,
                                              pad_to_max_length = True,
                                              return_tensors = 'pt')
BATCH_SIZE = 4
train_y = np.array(dftrain['type'].tolist())       ##np.array
trainset = TrainDataset(train_input_dict, train_y) ##trainset參數如init
trainloader = DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True)  

In [33]:
BATCH_SIZE = 64
test_x = dftest['text'].tolist()
test_input_dict = tokenizer.batch_encode_plus(test_x,
                                             add_special_tokens = True,
                                             max_length = 512,
                                             truncation = True,
                                             return_special_tokens_mask = True,
                                             pad_to_max_length = True,
                                             return_tensors = 'pt')
test_y = np.array(dftest['type'].tolist())
testset = TrainDataset(test_input_dict, test_y)
testloader = DataLoader(testset, batch_size = BATCH_SIZE, shuffle = True)

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## 類別對應 0：政治 1：生活 2：國際 3：體育 4：娛樂 5：社會 6：財經
def get_test_acc(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           token_type_ids = segment_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            pred = torch.argmax(outputs[1],dim=-1)
            total += labels.size()[0]
            correct += (pred == labels).sum().item()
    
    return correct/total

In [ ]:
## BertForSequencelassification
num_labels = 3
model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model = model.to(device)
model.train()

# loss_fn = F.CrossEntropyLoss()

EPOCHS = 3
# type_weight = torch.FloatTensor([2.1463606e+00, 2.6185270e+00, 6.5702598e+00]).to(device)
# type_loss_func = F.CrossEntropyLoss(weight=type_weight)
optimizer = AdamW(model.parameters(),lr = 5e-6)
# optimizer = torch.optim.Adam(binary_model.parameters(), lr=2e-5)

for epoch in range(0,EPOCHS):
    i = 0
    running_loss = 0.0
    correct = 0
    total = 0

    for (i,data) in enumerate(trainloader):
        
        tokens_tensors ,  segments_tensors , masks_tensors , labels  = [t.to(device) for t in data]
        
        bert_outputs = model(input_ids=tokens_tensors, 
                             token_type_ids=segments_tensors, 
                             attention_mask=masks_tensors,
                             labels = labels)
        
        optimizer.zero_grad()
        logits = bert_outputs[1]
        ##權重
#         weight_loss = type_loss_func(logits,labels)
        pred = torch.argmax(logits,dim = -1)
#         weight_loss.zero_grad()
        loss = bert_outputs[0] 
#         loss += weight_loss
        loss.backward()
        optimizer.step()
        
        total += pred.size()[0]
        correct += (pred == labels).sum().item()
        running_loss += loss.item()
        print(f'\rEpoch [{epoch+1}/{EPOCHS}] {i}/{len(trainloader)} Loss: {running_loss:.4f} Acc : {(correct/total):.4f}', end='')
    test_acc = get_test_acc(model,testloader)
    ##all save
    torch.save(model.state_dict(),'./BERT_3class_4e6_' + str(epoch) + '.pkl')
    print('test acc:' , test_acc)


Some weights of the model checkpoint at ./bert_pretrain_news/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

Epoch [1/3] 2893/2894 Loss: 516.8821 Acc : 0.9390test acc: 0.7304804804804805
Epoch [2/3] 2893/2894 Loss: 264.5359 Acc : 0.9671test acc: 0.7012012012012012
Epoch [3/3] 1896/2894 Loss: 90.0948 Acc : 0.9823